# Reflection and Blogpost Writing

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

/Users/vansh/Desktop/Agents/venv/lib/python3.12/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


[autogen.oai.client: 12-05 11:11:48] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Mastering AI with DeepLearning.AI"

Looking to dive into the world of artificial intelligence and machine learning? Look no further than DeepLearning.AI! Founded by AI expert Andrew Ng, this platform offers top-notch courses in deep learning to help you upskill and stay ahead in this rapidly evolving field. From beginner to advanced levels, DeepLearning.AI provides comprehensive content designed to cater to your learning needs. Whether you're a tech enthusiast or a professional looking to expand your skill set, DeepLearning.AI equips you with the knowledge and tools to succeed in the age of AI. Embark on your AI journey today!


In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

[autogen.oai.client: 12-05 11:18:09] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Mastering AI with DeepLearning.AI"

Looking to dive into the world of artificial intelligence and machine learning? Look no further than DeepLearning.AI! Founded by AI expert Andrew Ng, this platform offers top-notch courses in deep learning to help you upskill and stay ahead in this rapidly evolving field. From beginner to advanced levels, DeepLearning.AI provides comprehensive content designed to cater to your learning needs. Whether you're a tech enthusiast or a professional looking to expand your skill set, DeepLearning.AI equips you with the knowledge and tools to succeed in the age of AI. Embark on your AI journey today!

--------------------------------------------------------------------------------
Critic (to Writ

# nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 12-05 11:21:46] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 12-05 11:21:52] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

[autogen.oai.client: 12-05 11:21:58] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 12-05 11:22:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Mastering AI with DeepLearning.AI"

Looking to dive into the world of artificial intelligence and machine learning? Look no further than DeepLearning.AI! Founded by AI expert Andrew Ng, this platform offers top-notch courses in deep learning to help you upskill and stay ahead in this rapidly evolving field. From beginner to advanced levels, DeepLearning.AI provides comprehensive content designed to cater to your learning needs. Whether you're a tech enthusiast or a professional looking to expand your skill set, DeepLearning.AI equips you with the knowledge and tools to succeed in the age of AI. Embark on your AI journey today!

--------------------------------------------------------------------------------

**************

In [15]:
print(res.summary)

Title: "Master AI with DeepLearning.AI: Courses by Andrew Ng"

Delve into the world of artificial intelligence and machine learning with AI Deep Learning Courses by Andrew Ng on DeepLearning.AI. Explore top-notch content covering artificial intelligence courses, machine learning training, and online AI education by experts. Whether you're a tech enthusiast or a professional seeking to upskill, start mastering AI with DeepLearning.AI today. Stay ahead of the curve and propel your career in technology. Optimize your digital journey by enrolling in courses tailored to your learning needs. Begin your AI adventure now and join the ranks of the tech-savvy! Start mastering AI with DeepLearning.AI today and stay at the forefront of technology!
